In [3]:
import numpy as np
import pandas as pd 
import string
import random
import csv
import time
import math
import multiset 
import plotly.express as px
import json 

## Reading Data Files 

In [4]:
# The target file includes the original sequences that encodes the data. 
with open('targetBlawat et al.LDPC.txt') as f:
    reads = f.readlines()
for i in range(0,len(reads)):
    reads[i] = reads[i][0:len(reads[i])-1]
# reads 

In [5]:
np.size(reads)

9608

In [6]:
# The errors_shuffled file include the erroneous copies shuffled. 
with open('errors_shuffled.txt') as f:
    reads_err = f.readlines()
for i in range(0,len(reads_err)):
#     reads_err[i] = reads_err[i][0:len(reads_err[i])-1]
    reads_err[i] = reads_err[i].strip()
# reads_err

In [7]:
np.size(reads_err)

1161706

In [8]:
# evyat.txt include the erroneous copies clustered and labeled. 
with open('evyat.txt') as f:
    reads_cl = f.readlines()
for i in range(0,len(reads_cl)):
    reads_cl[i] = reads_cl[i].strip()
# reads_cl

# Functions


In [9]:
def hash_fun_ind(x,a,w,l): 
    cnt = 0 
    for i in range(0,len(x)):
        if x[i] == a[cnt]:
            cnt += 1  
        else:
            cnt = 0 
        if cnt == w:
            return i-w+1 
    return -1

def hash_fun(x,a,w,l):
    ind = hash_fun_ind(x,a,w,l)
    return x[ind:min(len(x),ind+w+l)]

def ind_st(st):
    #All 3-grams: AAA,AAG,AAC,AAT,AGA,AGC,...,TTA,TTG,TTC,TTT
    # A-0, G-1, C-2, T-3
    # index of CAT = Decimal representaion of (203)
    N_q = {"A":0,"C":1,"G":2,"T":3}
    dec = 0
    for i in range(0,len(st)):
        dec += N_q[st[i]]*(4**(len(st)-i-1))
    return dec 
    
def bin_sig(x,q):
    bs = [0]*(4**q) 
    for i in range(0,len(x)-q+1):
        st = x[i:i+q]
        bs[ind_st(st)] = 1
    return bs 
        
def ham_dis(x,y):
    dis = 0
    for i in range(0,len(x)):
        if(x[i]!= y[i]):
            dis += 1
    return dis 

def rand_perm(w):
    return ''.join(random.choice('ACGT') for _ in range(w))

def merge(x,y,parent):
    parent[x]= min(parent[x],parent[y])
    parent[y]= min(parent[x],parent[y])
    return parent
    
def rep_find(inp,parent):
    temp = inp
    cnt = 0
    while(parent[temp]!=temp and cnt<10):
        cnt += 1
        temp=parent[temp]
    return temp 

def edit_dis(s1, s2):
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

def find_in_arr(el,parent):
    cnt = 0
    for i in range(0,len(parent)):
        if parent[i] == el: 
            cnt += 1 
    return cnt 
        
def readFile(fileName):
        fileObj = open(fileName, "r") #opens the file in read mode
        words = fileObj.read().splitlines() #puts the file into an array
        fileObj.close()
        for i in range(0,len(words)):
            words[i] = int(words[i])
        return words

def read_file_dict(array):
    d = dict(enumerate(array.flatten()))
    return d[0]
    
def display_parent(parent):
    clstr = {}
    for i in range(0,len(parent)):
        clstr[i]=[]
    for i in range(0,len(parent)):
        clstr[rep_find(i,parent)].append(i)
    return clstr

def display_parent_ver2(parent,C_til_prev,parent_prev):
    clstr = C_til_prev
    for i in range(0,len(parent)):
        if(parent[i]!=parent_prev[i]):
#                 print(i,rep_find(i,parent_prev),clstr[rep_find(i,parent_prev)])
            clstr[rep_find(i,parent)].extend(clstr[i])
            clstr[rep_find(i,parent_prev)] = []
    return clstr

In [10]:
q =  3
x = "AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA"
y = "AACACGAAATAACACACCACGAAGAGACACGACACAACACAACAGAGCACAACAGAGCACAACAC"
ham_dis(bin_sig(x,q),bin_sig(y,q))

23

In [11]:
q

3

In [12]:
edit_dis(x,y)

28

In [13]:
x = reads_err[1]
y = reads_err[215]
ham_dis(bin_sig(x,q),bin_sig(y,q))

7

In [14]:
edit_dis(x,y)

4

# Clustering Algorithm

In [15]:
def rep_in_C(read,C_reps):
    lower = 0;
    upper = len(C_reps) - 1;
    while (lower <= upper):
        mid = lower + int((upper - lower) / 2)
#         print(upper,mid)
        res = -1
        if (read == (C_reps[mid][0])):
            return C_reps[mid][1]
        if (read > (C_reps[mid][0])):
            lower = mid + 1;
        else:
            upper = mid - 1;
    return -1;

def comp_clstrs(alg_clstr,org_clstr,gamma):
    num_exist = 0
    if(len(alg_clstr)>len(org_clstr)):
#         print(alg_clstr)
        return 0 
    else:
        for i in range(0,len(alg_clstr)):
            flg_exist = 0
            for j in range(0,len(org_clstr)):
                if(reads_err[alg_clstr[i]] == org_clstr[j]):
                    flg_exist = 1
                    num_exist +=1
                    break
            if(flg_exist == 0):
                return 0
        if(num_exist < gamma*len(org_clstr)):
            return 0
            
        return 1 
    
def calc_acrcy(clustering,C_dict,C_reps,gamma):
#     clustering = display_parent(parent)
    acrcy = 0
    for i in range(0,len(clustering)):
        if(len(clustering[i])>=1):
            acrcy = acrcy + comp_clstrs(clustering[i],C_dict[rep_in_C(reads_err[clustering[i][0]],C_reps)],gamma)
    return acrcy

def min_max(val1,val2):
    min_val = min(val1,val2)
    max_val = max(val1,val2)
    return min_val,max_val
    

In [16]:
# (Read, Cluster rep of the cluster to which the read belongs to)
C_reps = []
C_dict = {}
rep = reads_cl[0]
for i in range(1,len(reads_cl)):
    if reads_cl[i] != "":
        if reads_cl[i][0] == "*":
            if(len(C_reps)>0):
                C_dict[rep].pop()
                C_reps.pop()
            rep=reads_cl[i-1]
            C_dict[rep]=[]
        else:
#             print(C_dict)
            C_dict[rep].append(reads_cl[i])
            C_reps.append((reads_cl[i],rep))
C_reps.sort(key=lambda x:x[0])

In [17]:
C_reps[0:5]

[('AAAAAAACCGGAATATCAATCGAGCGAATGGGATACTACATTAAGGGATATCACTTGATCACAAG',
  'AACGAAACCGGAATATCAATCGAGCGAATGGGATACTACATTAAGGGATATCACTTGATCACAAG'),
 ('AAAAAAAGGTGAACTCGAAGACGTTAACGATATGACCCGTCAATTAAATGACGTATACACTACT',
  'AACTAAAGGTGAACTCGAAGATCGTTAACGATATGACCCGTCAATTAAATGACGTATACACTACT'),
 ('AAAAACACACTCAACGATACCTCTAACGGATAATCCGGTCACTAACATAAGAAGCACGCAAT',
  'AACTACAAACACACTCAACGATACTCTAACGGATAATCCGGTCACTAACATAAGAAGCACGCAAT'),
 ('AAAAACACCAATCAGATAAGAGTATAAGAGATAAGAACATCTACGAATCCACGCAATATGACG',
  'AACCAAACACCAATCAGATAAGAGTATAAGAGATAAGAACATCTACGAATCCACGCAATATGACG'),
 ('AAAAACACTTAGAAACATCAAGAGCAGCATATCTACCTCGAGTCCGTCTGCACACAACGTAGC',
  'AACTAAACACTTAGAAACATCAAGAGCAGCATATCAACCTCGAGTCCGTCTGCACACAACGTAGC')]

In [18]:
C_dict[list(C_dict.keys())[0]][0:5]

['AATCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTACAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA',
 'AACCTCTAGACAACACAACTATTAGTACCTATTATAAACACAACATAAGGTCAAGATAAGCAGAGA']

In [19]:
# Precompute all the binary signatures
q = 3
bin_sig_arr  = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    bin_sig_arr[i]=bin_sig(reads_err[i],q)

In [22]:
reads_err_ind  = [0]*(len(reads_err))
parent = [0]*(len(reads_err))
for i in range(0,len(reads_err)):
    reads_err_ind[i] = (i,reads_err[i]) 
    parent[i] = i

C_til = display_parent(parent)
acrcy_dict = {}
time_itr_dict = {}
# parent_dict = {}
# parent_dict[0] = parent.copy() 

DESIRED_ACRCY = 0.99
gamma = 0.9
card_C = len(reads)
local_comm = 800

q = 3
th_high = 12
th_low  = 8
r = 6
w = math.ceil(math.log(len(reads_err[0]),4))
l = math.ceil(math.log(len(reads_err),4))

lcl_step = 0
acrcy = 0

for lcl_step in range(0,local_comm):
# while(acrcy < DESIRED_ACRCY):   
    time_start = time.time()

    hash_select = [0]*(len(reads_err))    
    for i in range(0,len(C_til)):
        if(len(C_til[i])!=0):    
            hash_select[random.choice(C_til[i])]=1
            
    a = rand_perm(w)
    hash_C_til = [0]*(len(reads_err))
    for i in range(0,len(reads_err_ind)):
        if(hash_select[i]==0):
            hash_C_til[i] = (i,"")
        else:
            hash_C_til[i] = (i,hash_fun(reads_err_ind[i][1],a,w,l))    
    hash_C_til.sort(key=lambda x:x[1])

    cnt = 0
    for i in range(0,len(hash_C_til)-1):
        if(hash_C_til[i][1] == ""):
            continue 
        else:
            if hash_C_til[i][1] == hash_C_til[i+1][1]:
#                 if rep_find(i,parent) == rep_find(i+1,parent):
#                     continue 
                x = reads_err[hash_C_til[i][0]]
                y = reads_err[hash_C_til[i+1][0]]
                if((ham_dis(bin_sig_arr[hash_C_til[i][0]],bin_sig_arr[hash_C_til[i+1][0]]) <= th_low) or ((ham_dis(bin_sig_arr[hash_C_til[i][0]],bin_sig_arr[hash_C_til[i+1][0]]) <= th_high) and edit_dis(x,y)<=r)):
                    cnt +=  1                   
                    min_temp,max_temp = min_max(rep_find(hash_C_til[i][0],parent),rep_find(hash_C_til[i+1][0],parent))                    
                    C_til[min_temp].extend(C_til[max_temp])
                    C_til[max_temp] = []        
                    parent[max_temp] = min_temp                    

    acrcy = calc_acrcy(C_til,C_dict,C_reps,gamma)/card_C
    acrcy_dict[lcl_step+1] = acrcy
    
#     parent_dict[lcl_step+1]=parent.copy()
    
    time_itr = time.time() - time_start
    time_itr_dict[lcl_step+1]=time_itr
    
    print(acrcy)
    print(time_itr)
    print(lcl_step,cnt)
    
#     lcl_step += 1

0.004787676935886761
27.161977291107178
0 104573
0.026852622814321398
31.398178815841675
1 223233
0.043401332223147376
21.672237634658813
2 52456
0.08357618651124063
24.356590509414673
3 123116
0.13218151540383014
23.145870208740234
4 105196
0.15705661948376354
16.653796672821045
5 45393
0.18952955870108243
16.499675989151
6 44603
0.19816819317235637
13.324878692626953
7 409
0.2760199833472107
20.97982907295227
8 104783
0.34908409658617817
17.421878337860107
9 70994
0.3894671107410491
12.619071245193481
10 33060
0.4317235636969192
12.505192041397095
11 31289
0.49427560366361367
13.97725772857666
12 42093
0.5289342214820982
10.524420022964478
13 15460
0.5570358034970858
10.094012022018433
14 16818
0.5629683597002498
8.202784061431885
15 3443
0.5974188176519567
9.917822360992432
16 16439
0.6789134054954205
12.60509705543518
17 35982
0.6789134054954205
6.968827724456787
18 10
0.7375104079933389
10.270014762878418
19 19844
0.7386552872606161
6.579463720321655
20 93
0.7497918401332223
7.384

KeyboardInterrupt: 

In [23]:
with open('time_itr_dict_12,8,6,200,0.9_clstr_dict.json', 'w') as fp:
    json.dump(time_itr_dict, fp)
with open('acrcy_dict_12,8,6,200,0.9_clstr_dict.json', 'w') as fp:
    json.dump(acrcy_dict, fp)

In [25]:
total_time = 0
for i in range(1,len(time_itr_dict)+1):
    total_time += time_itr_dict[i]
total_time

575.1226627826691

In [26]:
acrcy_dict_per = {}
for i in range(1,len(acrcy_dict)+1):
    acrcy_dict_per[i] = acrcy_dict[i]*100
keys = acrcy_dict_per.keys()
values = acrcy_dict_per.values()

px.line(x=keys,y=values)

In [ ]:
keys = time_itr_dict.keys()
values = time_itr_dict.values()

px.line(x=keys,y=values)

In [27]:
!git status 

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Clustering-algorithm-single-core-clstr-diction-(120,0.9).ipynb
	modified:   Clustering-algorithm-single-core-parent-array-(120,0.9).ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	Clustering-algorithm-accuracy-measure.ipynb
	Clustering-algorithm-single-core-clstr-diction-(120,0.9)-Copy1.ipynb
	acrcy_dict_12,8,6,200,0.9_clstr_dict.json
	acrcy_dict_12,8,6,200,0.9_par_arr.json
	errors_shuffled.txt
	evyat.txt
	nips-2017-camera-long.pdf
	targetBlawat et al.LDPC.txt
	time_itr_dict_12,8,6,200,0.9_clstr_dict.json
	time_itr_dict_12,8,6,200,0.9_par_arr.json
	../Scaling of Errors with cluster size/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
# !git add Clustering-algorithm-single-core-clstr-diction-\(120,0.9\).ipynb

In [ ]:
# !git add -u && git status && git commit -m "acrcy plot" && git push origin master 